In [1]:
using Plots
using DifferentialEquations

In [4]:
arr(ao, ea, T) = ao*exp(ea/8.314/T) #need to be in units of J/mol and K

arr (generic function with 1 method)

We begin with the equations for the rates:
$$\frac{\partial